In [ ]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (7,7)

import random
import tensorflow as tf

seed = 42


random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# Import deepscore model # Add deepscore folder to path
from deepscore import deepscore 
import pickle

2025-09-02 16:01:28.062424: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 16:01:28.215952: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-02 16:01:28.216960: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:

# LOAD the PROCESS the REFERENCE

# ## Preprocessing


cell_type = 'subclass.l1'

ref_py = sc.read('/home/macera/Documentos/CZI/MANUSCRIPT_PREP/HORIZONTAL_RNA/objects/local.h5ad')  # LOAD the HCA Atlas
ref_py = ref_py.raw.to_adata()
ref_py.layers['counts'] = ref_py.X.copy()

ref_py.var['ENSG'] = ref_py.var.index.copy()
ref_py.var.index = ref_py.var['feature_name'].copy()

# SUBSET THE ATLAS TO MATCH OUR SAMPLE BIOLOGY

cortex_celltypes_l1= ['DCT',
 'PEC',
 'CNT',
 'POD',
 'PT',
 'IC',
 'IMM',
 'NEU',
 'VSM/P',
 'TAL',
 'EC',
 'FIB',
 'PC']

cortex_celltypes_l3= ['DCT1',
 'DCT2',
 'IC-B',
 'B',
 'MD',
 'CNT-IC-A',
 'MC',
 'CNT',
 'PT-S1/2',
 'CCD-IC-A',
 'PEC',
 'pDC',
 'EC-GC',
 'POD',
 'VSMC',
 'aPT',
 'CNT-PC',
 'FIB',
 'cDC',
 'REN',
 'EC-PTC',
 'T',
 'PT-S3',
 'ncMON',
 'NKC/T',
 'aTAL1',
 'C-TAL',
 'PL',
 'CCD-PC',
 'SC/NEU',
 'EC-LYM',
 'MDC',
 'N',
 'MAST',
 'aTAL2',
 'aFIB',
 'MYOF',
 'MAC-M2',
 'EC-AEA',
 'VSMC/P']


ref_py = ref_py[ref_py.obs['subclass.l1'].isin(cortex_celltypes_l1)]

ref_py = ref_py[ref_py.obs['subclass.l3'].isin(cortex_celltypes_l3)]

ref_py = ref_py[ref_py.obs['condition.long'].isin(['Normal Reference'])]


ref_py = ref_py[ref_py.obs['state.l2'].isin(['reference','adaptive - epi','adaptive - str'])].copy()


cell_type = 'subclass.l1'
overlapping = False
compute = False


ref_py.X = ref_py.X.copy()
sc.pp.normalize_total(ref_py, target_sum=1e4)
sc.pp.log1p(ref_py)


markers_filename= f'HCA_l1'

# Identify HCA ATLAS differentially expressed genes between cell types 

if compute == True:
    sc.tl.rank_genes_groups(ref_py, cell_type, method='wilcoxon', use_raw=False)
    ranked_genes_populations = ref_py.uns['rank_genes_groups'].copy()
    with open(f'/home/macera/Documentos/CZI/MANUSCRIPT_PREP/DEEPSCORE/markers_ds/{markers_filename}.pickle', 'wb') as handle:
        pickle.dump(ranked_genes_populations, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'/home/macera/Documentos/CZI/MANUSCRIPT_PREP/DEEPSCORE/markers_ds/{markers_filename}.pickle', 'rb') as handle:
        ranked_genes_populations = pickle.load(handle) 

In [ ]:
import os
## SET PARAMETERS
n_markers = 500 # Max number of markers to use per cell-type
overlapping = False # Parameter to control overlapping marker genes between cell types on the prediction.


ref_py_save = ref_py.copy()
mod = 'RECK'
if os.path.exists(f'csv/Deepscore_HCA_l1_{mod}_CLEAN.csv'):
    print(f'{mod} already exists!')

adata = sc.read(f'/home/macera/Documentos/CZI/MANUSCRIPT_PREP/REVIEWS/external_data/reck/rna_ready.h5ad', compression='gzip')
# adata = adata.raw.to_adata()
adata.layers['counts'] = adata.X.copy()


In [9]:
with open(f'markers_ds/{markers_filename}.pickle', 'rb') as handle:
    ranked_genes_populations = pickle.load(handle) 

if overlapping:
    selected_markers =[]
    for cell_type_ in ref_py.obs[cell_type].unique():
        cell_type_markers = []
        for marker in ranked_genes_populations['names'][cell_type_][:n_markers]:
            if marker in adata.var.index: 
                selected_markers.append(marker)
    selected_markers = set(selected_markers)

else:
    # Step 2: Store markers for each subset
    subset_markers_dict ={}
    for subset in ref_py.obs[cell_type].unique():
        subset_markers = ranked_genes_populations['names'][subset]
        subset_markers = [gene for gene in subset_markers if gene in adata.var.index]
        subset_markers_dict[subset] = set(subset_markers[:n_markers+100])

    # Step 3: Identify overlapping markers
    overlapping_markers = set()
    for subset, markers in subset_markers_dict.items():
        for other_subset, other_markers in subset_markers_dict.items():
            if subset != other_subset:
                overlapping_markers.update(markers.intersection(other_markers))

    # Step 4: Select markers for each subset, excluding overlapping markers
    marker_dict = {}
    for subset, markers in subset_markers_dict.items():
        unique_markers = [marker for marker in markers if marker not in overlapping_markers]
        marker_dict[subset] = unique_markers[:n_markers]  # Select up to TOP n_markers
    selected_markers = [marker for subset in marker_dict for marker in marker_dict[subset]]



sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)


# Subset the data to the selected markers

ref_py = ref_py_save[:, list(selected_markers)].copy()
adata = adata[:, list(selected_markers)].copy()

len(selected_markers)

sc.pp.scale(ref_py)
sc.pp.scale(adata)

ref_py.obs[cell_type] = ref_py.obs[cell_type].tolist()
len(ref_py.obs[cell_type].unique())


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


n_feat = ref_py.shape[1]
n_labs = len(ref_py.obs[cell_type].unique())

ds = deepscore.DeepScore(hidden_nodes=[1024, 256],
                n_features=n_feat, 
                n_labels=n_labs,
                epochs=30,
                batch_size=128, 
                activation="relu", 
                dropout=True, 
                dropout_rate=0.1,
                batchnorm=True, 
                lr=0.001,
                weight_reg=True)




import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

ds.set_reference(ref_py, label_by=cell_type, test_prop=0.1)

ds.train(earlystopping=True, patience=10, lr_scheduler=scheduler,)
# ds.model.save(f'models/deepscore') # In case you want to save the DS model

prob_df, adata = ds.annotate(adata, pred_key='Deepscore_HCA',Unclassified = False,return_pred_matrix=True)

# SAVE the RESULTS on csv
adata.obs[['Deepscore_HCA','Deepscore_HCA_score']].to_csv(f'csv/Deepscore_HCA_l1_{mod}_CLEAN.csv')

prob_df.to_csv(f'csv/prob_matrix/Deepscore_HCA_l1_{mod}_CLEAN.csv')


Model: "deepscore"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 2550)              10200     
 Normalization)                                                  
                                                                 
 dense1024 (Dense)           (None, 1024)              2612224   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 batch_normalization_1 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense256 (Dense)            (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0 

2025-09-02 15:44:37.757842: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 584194800 exceeds 10% of free system memory.
2025-09-02 15:44:38.191204: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 525769200 exceeds 10% of free system memory.


Epoch 1/30
403/403 [==============================] - 15s 34ms/step - loss: 0.1792 - categorical_accuracy: 0.9458 - val_loss: 0.1151 - val_categorical_accuracy: 0.9619 - lr: 0.0010
Epoch 2/30
403/403 [==============================] - 13s 32ms/step - loss: 0.0459 - categorical_accuracy: 0.9846 - val_loss: 0.1220 - val_categorical_accuracy: 0.9640 - lr: 0.0010
Epoch 3/30
403/403 [==============================] - 13s 33ms/step - loss: 0.0266 - categorical_accuracy: 0.9910 - val_loss: 0.1531 - val_categorical_accuracy: 0.9578 - lr: 0.0010
Epoch 4/30
403/403 [==============================] - 13s 32ms/step - loss: 0.0200 - categorical_accuracy: 0.9932 - val_loss: 0.1529 - val_categorical_accuracy: 0.9612 - lr: 0.0010
Epoch 5/30
403/403 [==============================] - 12s 30ms/step - loss: 0.0172 - categorical_accuracy: 0.9942 - val_loss: 0.1607 - val_categorical_accuracy: 0.9619 - lr: 0.0010
Epoch 6/30
403/403 [==============================] - 13s 32ms/step - loss: 0.0184 - categorica

2025-09-02 15:46:59.583749: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 478961400 exceeds 10% of free system memory.


 13/367 [>.............................] - ETA: 1s 

2025-09-02 15:46:59.845977: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 478961400 exceeds 10% of free system memory.


367/367 [==============================] - 4s 10ms/step
